In [1]:
!pip install msgpack xgboost

You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install bayesian-optimization

You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import xgboost as xgb
import pandas as pd
import numpy as np
import warnings
import boto3
import io
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import RobustScaler, Imputer, StandardScaler
from sklearn.metrics import mean_squared_error, f1_score, confusion_matrix,accuracy_score, recall_score, precision_score
from sklearn import metrics
from bayes_opt import BayesianOptimization
import matplotlib.pyplot as plt
from itertools import chain, zip_longest
bucket = 'boi-banregio'
target = 'BMI'
s3_bucket_resource = boto3.resource('s3').Bucket(bucket)

In [4]:
# Aquí colocamos la ruta de nuestro directorio, el nombre de nuestro archivo y si tiene o no cabecera
prefix = 'datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION'

In [5]:
model_features_file_name = "MODEL_DATASET/MODEL_FEATURES+EXTRAS.csv"
model_features_file_key = "{}/{}".format(prefix, model_features_file_name)
model_features_obj = s3_bucket_resource.Object(model_features_file_key).get()
model_features_data = pd.read_csv(io.BytesIO(model_features_obj['Body'].read()))

In [6]:
target_file_name = "TARGETS.csv"
target_file_key = "{}/{}".format(prefix, target_file_name)
target_obj = s3_bucket_resource.Object(target_file_key).get()
target_data = pd.read_csv(io.BytesIO(target_obj['Body'].read())).loc[:,[target]]

In [7]:

display(model_features_data.describe())
serie=pd.Series((3*np.std(model_features_data)))
model_features_data.clip(upper=serie,inplace=True, axis=1)





,AVG_R_BAL_BY_CRED_LIM_12M,DEUDA_TOT,MIN_R_BAL_BY_MAX_BAL_03M,MAX_D_CRED_LIM_TO_BAL_03M,AVG_D_CRED_LIM_TO_BAL_06M,AVG_PND_MENOS_R_BAL_BY_MAX_BAL_06M,AVG_R_BAL_BY_MAX_BAL_12M,AVG_PND_MENOS_BAL_12M,BK12_MAX_CREDIT_AMT,AVG_PND_MAS_BAL_12M,...,MAX_AVG2_VP_MES_U06M_2,MIN_AVG_HI_CALCAR_CEIL_12M,AVG_PND_MAS_MAX_HI_CALCAR_L_12M,MAX_MS_CL_BRG_R_CL_ACC,AVG_PND_MAS_MAX_HI_CALCAR_CEIL_12M,SUM_PAGOREALIZADO_PERIODO,DIAS_ATRASO,DPCV_CAP,DPCV_INT,DPCV_MIN
count,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,...,4.252870e+05,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000
mean,0.012022,3.243717,-0.483808,0.672008,0.683300,-0.341297,-0.251894,0.483345,3.288619,0.482477,...,5.012855e+02,0.855755,0.223056,2.168418,1.402803,6.400931,1.938785,-0.179172,-0.298515,-0.179172
std,23.915491,350.995352,1.100280,1.825779,1.946918,0.941598,0.768336,1.180457,492.157345,1.177261,...,9.149831e+04,6.743288,0.885903,16.910407,5.052671,124.400862,11.199988,0.785282,0.680283,0.785282
min,-1.789694,-0.663866,-3.601348,-10.698307,-11.800963,-4.373871,-3.104854,-0.371077,-0.967742,-0.371139,...,-9.101890e-02,-0.944444,-1.377633,-31.000000,-1.123769,-0.255973,0.000000,-19.290323,-21.344828,-19.290323
25%,-0.576496,-0.386555,-0.733688,-0.230336,-0.234195,-0.648744,-0.635807,-0.250403,-0.419355,-0.253856,...,-4.634485e-02,-0.944444,-0.385737,0.000000,-0.275705,-0.224162,0.000000,-1.000000,-0.724138,-1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.423504,0.613445,0.266312,0.769664,0.765805,0.351256,0.364193,0.749597,0.580645,0.746144,...,9.536552e-01,0.055556,0.614263,0.000000,0.724295,0.775838,0.000000,0.000000,0.275862,0.000000
max,7368.944220,56249.680672,0.359358,103.564446,131.391148,0.439008,0.486416,20.991531,96773.225806,14.555473,...,2.233703e+07,164.222222,4.188006,208.000000,70.338397,24488.298307,598.000000,0.935484,0.275862,0.935484


In [8]:
display(model_features_data.describe())

imputer = Imputer(missing_values='NaN', strategy='median', axis=0)
imputer = imputer.fit(model_features_data)
model_features_data.loc[:,:] = imputer.transform(model_features_data)

display(model_features_data.describe())

scaler = RobustScaler()
scaler.fit(model_features_data)
model_features_data.loc[:, :] = scaler.transform(model_features_data)

display(model_features_data.describe())
#df = df1.loc[:,features]
#display(df.describe())
#serie=pd.Series((3*np.std(df)))
#df.clip(upper=serie,inplace=True, axis=1)
#display(df.describe())

,AVG_R_BAL_BY_CRED_LIM_12M,DEUDA_TOT,MIN_R_BAL_BY_MAX_BAL_03M,MAX_D_CRED_LIM_TO_BAL_03M,AVG_D_CRED_LIM_TO_BAL_06M,AVG_PND_MENOS_R_BAL_BY_MAX_BAL_06M,AVG_R_BAL_BY_MAX_BAL_12M,AVG_PND_MENOS_BAL_12M,BK12_MAX_CREDIT_AMT,AVG_PND_MAS_BAL_12M,...,MAX_AVG2_VP_MES_U06M_2,MIN_AVG_HI_CALCAR_CEIL_12M,AVG_PND_MAS_MAX_HI_CALCAR_L_12M,MAX_MS_CL_BRG_R_CL_ACC,AVG_PND_MAS_MAX_HI_CALCAR_CEIL_12M,SUM_PAGOREALIZADO_PERIODO,DIAS_ATRASO,DPCV_CAP,DPCV_INT,DPCV_MIN
count,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,...,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000
mean,-0.115812,0.241655,-0.483808,0.593698,0.600267,-0.341297,-0.251894,0.438407,0.396470,0.438314,...,62.915855,0.444994,0.213346,1.858692,1.072314,4.199640,1.395097,-0.179172,-0.298515,-0.179172
std,0.978579,10.726220,1.100280,1.356867,1.401135,0.941598,0.768336,1.004731,10.053387,1.008060,...,3241.334732,3.017811,0.854687,15.412485,3.184593,27.772306,6.024416,0.785282,0.680283,0.785282
min,-1.789694,-0.663866,-3.601348,-10.698307,-11.800963,-4.373871,-3.104854,-0.371077,-0.967742,-0.371139,...,-0.091019,-0.944444,-1.377633,-31.000000,-1.123769,-0.255973,0.000000,-19.290323,-21.344828,-19.290323
25%,-0.576496,-0.386555,-0.733688,-0.230336,-0.234195,-0.648744,-0.635807,-0.250403,-0.419355,-0.253856,...,-0.046345,-0.944444,-0.385737,0.000000,-0.275705,-0.224162,0.000000,-1.000000,-0.724138,-1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.423504,0.613445,0.266312,0.769664,0.765805,0.351256,0.364193,0.749597,0.580645,0.746144,...,0.953655,0.055556,0.614263,0.000000,0.724295,0.775838,0.000000,0.000000,0.275862,0.000000
max,71.746390,1052.984817,0.359358,5.477332,5.840747,0.439008,0.486416,3.541365,1476.470300,3.531780,...,274494.617114,20.229839,2.657705,50.731162,15.157996,373.202147,33.599923,0.935484,0.275862,0.935484


,AVG_R_BAL_BY_CRED_LIM_12M,DEUDA_TOT,MIN_R_BAL_BY_MAX_BAL_03M,MAX_D_CRED_LIM_TO_BAL_03M,AVG_D_CRED_LIM_TO_BAL_06M,AVG_PND_MENOS_R_BAL_BY_MAX_BAL_06M,AVG_R_BAL_BY_MAX_BAL_12M,AVG_PND_MENOS_BAL_12M,BK12_MAX_CREDIT_AMT,AVG_PND_MAS_BAL_12M,...,MAX_AVG2_VP_MES_U06M_2,MIN_AVG_HI_CALCAR_CEIL_12M,AVG_PND_MAS_MAX_HI_CALCAR_L_12M,MAX_MS_CL_BRG_R_CL_ACC,AVG_PND_MAS_MAX_HI_CALCAR_CEIL_12M,SUM_PAGOREALIZADO_PERIODO,DIAS_ATRASO,DPCV_CAP,DPCV_INT,DPCV_MIN
count,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,...,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000
mean,-0.115812,0.241655,-0.483808,0.593698,0.600267,-0.341297,-0.251894,0.438407,0.396470,0.438314,...,62.915855,0.444994,0.213346,1.858692,1.072314,4.199640,1.395097,-0.179172,-0.298515,-0.179172
std,0.978579,10.726220,1.100280,1.356867,1.401135,0.941598,0.768336,1.004731,10.053387,1.008060,...,3241.334732,3.017811,0.854687,15.412485,3.184593,27.772306,6.024416,0.785282,0.680283,0.785282
min,-1.789694,-0.663866,-3.601348,-10.698307,-11.800963,-4.373871,-3.104854,-0.371077,-0.967742,-0.371139,...,-0.091019,-0.944444,-1.377633,-31.000000,-1.123769,-0.255973,0.000000,-19.290323,-21.344828,-19.290323
25%,-0.576496,-0.386555,-0.733688,-0.230336,-0.234195,-0.648744,-0.635807,-0.250403,-0.419355,-0.253856,...,-0.046345,-0.944444,-0.385737,0.000000,-0.275705,-0.224162,0.000000,-1.000000,-0.724138,-1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.423504,0.613445,0.266312,0.769664,0.765805,0.351256,0.364193,0.749597,0.580645,0.746144,...,0.953655,0.055556,0.614263,0.000000,0.724295,0.775838,0.000000,0.000000,0.275862,0.000000
max,71.746390,1052.984817,0.359358,5.477332,5.840747,0.439008,0.486416,3.541365,1476.470300,3.531780,...,274494.617114,20.229839,2.657705,50.731162,15.157996,373.202147,33.599923,0.935484,0.275862,0.935484


,AVG_R_BAL_BY_CRED_LIM_12M,DEUDA_TOT,MIN_R_BAL_BY_MAX_BAL_03M,MAX_D_CRED_LIM_TO_BAL_03M,AVG_D_CRED_LIM_TO_BAL_06M,AVG_PND_MENOS_R_BAL_BY_MAX_BAL_06M,AVG_R_BAL_BY_MAX_BAL_12M,AVG_PND_MENOS_BAL_12M,BK12_MAX_CREDIT_AMT,AVG_PND_MAS_BAL_12M,...,MAX_AVG2_VP_MES_U06M_2,MIN_AVG_HI_CALCAR_CEIL_12M,AVG_PND_MAS_MAX_HI_CALCAR_L_12M,MAX_MS_CL_BRG_R_CL_ACC,AVG_PND_MAS_MAX_HI_CALCAR_CEIL_12M,SUM_PAGOREALIZADO_PERIODO,DIAS_ATRASO,DPCV_CAP,DPCV_INT,DPCV_MIN
count,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,...,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000
mean,-0.115812,0.241655,-0.483808,0.593698,0.600267,-0.341297,-0.251894,0.438407,0.396470,0.438314,...,62.915855,0.444994,0.213346,1.858692,1.072314,4.199640,1.395097,-0.179172,-0.298515,-0.179172
std,0.978579,10.726220,1.100280,1.356867,1.401135,0.941598,0.768336,1.004731,10.053387,1.008060,...,3241.334732,3.017811,0.854687,15.412485,3.184593,27.772306,6.024416,0.785282,0.680283,0.785282
min,-1.789694,-0.663866,-3.601348,-10.698307,-11.800963,-4.373871,-3.104854,-0.371077,-0.967742,-0.371139,...,-0.091019,-0.944444,-1.377633,-31.000000,-1.123769,-0.255973,0.000000,-19.290323,-21.344828,-19.290323
25%,-0.576496,-0.386555,-0.733688,-0.230336,-0.234195,-0.648744,-0.635807,-0.250403,-0.419355,-0.253856,...,-0.046345,-0.944444,-0.385737,0.000000,-0.275705,-0.224162,0.000000,-1.000000,-0.724138,-1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.423504,0.613445,0.266312,0.769664,0.765805,0.351256,0.364193,0.749597,0.580645,0.746144,...,0.953655,0.055556,0.614263,0.000000,0.724295,0.775838,0.000000,0.000000,0.275862,0.000000
max,71.746390,1052.984817,0.359358,5.477332,5.840747,0.439008,0.486416,3.541365,1476.470300,3.531780,...,274494.617114,20.229839,2.657705,50.731162,15.157996,373.202147,33.599923,0.935484,0.275862,0.935484


In [9]:
#ratio = float(np.sum(df1.loc[:, [target]] == 0)) / np.sum(df1.loc[:, [target]]==1)
# Comment out any parameter you don't want to test
def XGB_CV(
          max_depth,
          gamma,
          min_child_weight,
          max_delta_step,
          subsample,
          colsample_bytree,
          lambdas
         ):

    global AUCbest
    global ITERbest

#
# Define all XGboost parameters
#

    paramt = {
              'booster' : 'gbtree',
              'max_depth' : int(max_depth),
              'gamma' : gamma,
              'eta' : 0.1,
              'objective' : 'binary:logistic',
#              'nthread' : 4,
              'silent' : True,
              'eval_metric': 'auc',
              'subsample' : max(min(subsample, 1), 0),
              'colsample_bytree' : max(min(colsample_bytree, 1), 0),
              'min_child_weight' : min_child_weight,
              'max_delta_step' : int(max_delta_step),
              'scale_pos_weight' : 50,
              'lambda' : lambdas,
              'seed' : 1
              }

    folds = 10
    cv_score = 0

    print("\n Search parameters (%d-fold validation):\n %s" % (folds, paramt), file=log_file )
    log_file.flush()

    xgbc = xgb.cv(
                    paramt,
                    dtrain,
                    num_boost_round = 100,
                    stratified = True,
                    nfold = folds,
#                    verbose_eval = 10,
                    early_stopping_rounds = 100,
                    metrics = 'auc',
                    show_stdv = True
               )

    val_score = xgbc['test-auc-mean'].iloc[-1]
    train_score = xgbc['train-auc-mean'].iloc[-1]
    print(' Stopped after %d iterations with train-auc = %f val-auc = %f ( diff = %f ) train-gini = %f val-gini = %f' % ( len(xgbc), train_score, val_score, (train_score - val_score), (train_score*2-1),(val_score*2-1)) )
    if ( val_score > AUCbest ):
        AUCbest = val_score
        ITERbest = len(xgbc)

    return (val_score*2) - 1

In [ ]:
log_file = open('BaseUno_30+_full.log', 'a')
AUCbest = -1.
ITERbest = 0

X_train, X_test, y_train, y_test= train_test_split(model_features_data, target_data,test_size=0.2, random_state=123, stratify=target_data)
dtrain = xgb.DMatrix(data=X_train,label=y_train)

XGB_BO = BayesianOptimization(XGB_CV, {
                                     'max_depth': (2, 12),
                                     'gamma': (0.001, 10.0),
                                     'min_child_weight': (0, 20),
                                     'max_delta_step': (1, 100),
                                     'subsample': (0.4, 1.0),
                                     'colsample_bytree' :(0.4, 1.0),
                                     'lambdas' :(1,100)
                                    })

XGB_BO.explore({
              'max_depth':            [3, 8, 3, 8, 8, 3, 8, 3],
              'gamma':                [0.5, 8, 0.2, 9, 0.5, 8, 0.2, 9],
              'min_child_weight':     [0.2, 0.2, 0.2, 0.2, 12, 12, 12, 12],
              'max_delta_step':       [1, 20, 50, 70, 1, 20, 50, 90],
              'subsample':            [0.6, 0.8, 0.6, 0.8, 0.6, 0.8, 0.6, 0.8],
              'colsample_bytree':     [0.6, 0.8, 0.6, 0.8, 0.6, 0.8, 0.6, 0.8],
              'lambdas' :              [10,50,70,10,70,50,10,50]
              })

print('-'*130)
print('-'*130, file=log_file)
log_file.flush()

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    XGB_BO.maximize(init_points=10, n_iter=25, acq='ei', xi=0.0)

print('-'*130)
print('Final Results')
print('Maximum XGBOOST value: %f' % XGB_BO.res['max']['max_val'])
print('Best XGBOOST parameters: ', XGB_BO.res['max']['max_params'])
print('-'*130, file=log_file)
print('Final Result:', file=log_file)
print('Maximum XGBOOST value: %f' % XGB_BO.res['max']['max_val'], file=log_file)
print('Best XGBOOST parameters: ', XGB_BO.res['max']['max_params'], file=log_file)
log_file.flush()
log_file.close()

history_df = pd.DataFrame(XGB_BO.res['all']['params'])
history_df2 = pd.DataFrame(XGB_BO.res['all']['values'])
history_df = pd.concat((history_df, history_df2), axis=1)
history_df.rename(columns = { 0 : 'gini'}, inplace=True)
history_df['AUC'] = ( history_df['gini'] + 1 ) / 2
history_df.to_csv('BaseUno_30+_full-run-01-v1-grid.csv')


----------------------------------------------------------------------------------------------------------------------------------
Initialization
----------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   lambdas |   max_delta_step |   max_depth |   min_child_weight |   subsample | 
 Stopped after 100 iterations with train-auc = 0.940652 val-auc = 0.935972 ( diff = 0.004680 ) train-gini = 0.881303 val-gini = 0.871943
    1 | 08m16s |    0.87194 |             0.6000 |    0.5000 |   10.0000 |           1.0000 |      3.0000 |             0.2000 |      0.6000 | 
 Stopped after 100 iterations with train-auc = 0.992212 val-auc = 0.979763 ( diff = 0.012449 ) train-gini = 0.984424 val-gini = 0.959526
    2 | 33m17s |    0.95953 |             0.8000 |    8.0000 |   50.0000 |          20.0000 |      8.0000 |             0.2000 |      0.8000 | 
 Stop

In [ ]:
XGB_BO.res['max']['max_params']

In [28]:
X_train, X_test, y_train, y_test= train_test_split(model_features_data, target_data,test_size=0.2, random_state=123, stratify=target_data)
dtrain = xgb.DMatrix(data=X_train,label=y_train)

In [29]:
model_features_data.shape

(425287, 404)

In [30]:
xg_cl = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=1,
 max_depth=8,
 min_child_weight=19.50392156038251,
 gamma= 0.001,
 subsample=1,  # 0.8
 colsample_bytree=1,  # 0.4
 objective= 'binary:logistic',
 #nthread=4,
    reg_lambda=10,  # 17
 max_delta_step= 50,  # 100
 scale_pos_weight=24,  # 100
 early_stopping_rounds = 100,
 stratified = True,
 seed=1)


xg_cl.fit(X_train,y_train,eval_metric=['ams@0.15', 'auc'])



/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, early_stopping_rounds=100, gamma=0.001,
       learning_rate=0.1, max_delta_step=50, max_depth=8,
       min_child_weight=19.50392156038251, missing=None, n_estimators=1,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=10, scale_pos_weight=24, seed=1,
       silent=True, stratified=True, subsample=1)

In [13]:
dtrain = xgb.DMatrix(
    X_train, 
    label=y_train
)
dtest = xgb.DMatrix(
    X_test, 
    label=y_test
)

In [44]:
params = {
    'objective': 'binary:logistic', 
    'eta':0.2, 
    'max_depth': 8, 
    'eval_metric': 'auc',
    'min_child_weight': 19.50392156038251,
    'gamma': 0.001,
    'subsample': .8,  # 0.8
    'colsample_bytree': 0.4,  # 0.4
    # nthread=4,
    'reg_lambda': 20,  # 17
    'max_delta_step': 1,  # 100
    'scale_pos_weight': 100,  # 100
    'early_stopping_rounds': 100,
    'stratified': True,
    'seed': 1
}
num_round = 100

bst = xgb.train(params, dtrain, num_round)

[07:25:49] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 316 extra nodes, 0 pruned nodes, max_depth=8
[07:25:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 362 extra nodes, 0 pruned nodes, max_depth=8
[07:26:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 300 extra nodes, 0 pruned nodes, max_depth=8
[07:26:10] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 302 extra nodes, 0 pruned nodes, max_depth=8
[07:26:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 332 extra nodes, 0 pruned nodes, max_depth=8
[07:26:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 326 extra nodes, 0 pruned nodes, max_depth=8
[07:26:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 320 extra nodes, 0 pruned nodes, max_depth=8
[07:26:37] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 374 extra nodes, 0 pruned nodes, max_depth=8
[07:26:4

[07:33:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 0 pruned nodes, max_depth=8
[07:33:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 250 extra nodes, 0 pruned nodes, max_depth=8
[07:33:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 148 extra nodes, 0 pruned nodes, max_depth=8
[07:33:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 228 extra nodes, 0 pruned nodes, max_depth=8
[07:33:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 290 extra nodes, 0 pruned nodes, max_depth=8
[07:33:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 256 extra nodes, 0 pruned nodes, max_depth=8
[07:33:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 186 extra nodes, 0 pruned nodes, max_depth=8
[07:33:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 144 extra nodes, 0 pruned nodes, max_depth=8
[07:33:5

In [46]:
f1_sc = 0
max_step = 0
#thr_sample = (dtrain, y_train)
thr_sample = (dtest, y_test)
_score_preds = bst.predict(thr_sample[0])
for thr_step in np.linspace(0, 1, 101):
    _preds = predict(_score_preds, thr_step)
    f1_sc_step = f1_score(thr_sample[1], _preds)
    #print(thr_step, f1_sc_step)
    if f1_sc_step >= f1_sc:
        f1_sc = f1_sc_step
        max_step = thr_step
threshold = max_step
print('threshold', threshold)
print('max score', f1_sc)

threshold 0.93
max score 0.7505820721769498


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [39]:
# make predictions on datasets
train_pred = predict(bst.predict(dtrain), threshold)
val_pred = train_pred
test_pred = predict(bst.predict(dtest), threshold)

cm_train = confusion_matrix(y_train, train_pred)
cm_val = cm_train
cm_test = confusion_matrix(y_test, test_pred)

cm_train_pct = cm_train / cm_train.astype(np.float).sum()*100
cm_val_pct = cm_train_pct
cm_test_pct = cm_test / cm_test.astype(np.float).sum()*100

val_y=y_train
val_pred=train_pred

summary = [
    ['------', 'Train', 'Validation', 'Test'],
    ['confusion matrix', cm_train, cm_val, cm_test],
    ['confusion matrix pct', cm_train_pct, cm_val_pct, cm_test_pct],
    ['f1_score', f1_score(y_train, train_pred), f1_score(val_y, val_pred), f1_score(y_test, test_pred)],
    ['accuracy score', accuracy_score(y_train, train_pred), accuracy_score(val_y, val_pred), accuracy_score(y_test, test_pred)],
    ['recall score', recall_score(y_train, train_pred), recall_score(val_y, val_pred), recall_score(y_test, test_pred)],
    ['precision_score', precision_score(y_train, train_pred), precision_score(val_y, val_pred), precision_score(y_test, test_pred)]
]

# print pretty table
pretty_table(summary)

------              	Train                      	Validation                 	Test                       
confusion matrix    	[[323812   3035]           	[[323812   3035]           	[[80765   947]             
                    	 [   831  12551]]          	 [   831  12551]]          	 [  767  2579]]            
confusion matrix pct	[[95.17472056  0.89204624] 	[[95.17472056  0.89204624] 	[[94.9528557   1.11335794] 
                    	 [ 0.24424726  3.68898595]]	 [ 0.24424726  3.68898595]]	 [ 0.90173764  3.03204872]]
f1_score            	0.8665423916045292         	0.8665423916045292         	0.7505820721769498         
accuracy score      	0.9886370650356083         	0.9886370650356083         	0.9798490441816172         
recall score        	0.9379016589448513         	0.9379016589448513         	0.7707710699342498         
precision_score     	0.805273963813679          	0.805273963813679          	0.7314237095859331         


In [9]:
X_train_predictions = xg_cl.predict(X_train)
X_train_predprob = xg_cl.predict_proba(X_train)[:,1]

X_test_predictions = xg_cl.predict(X_test)
X_test_predprob = xg_cl.predict_proba(X_test)[:,1]


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [10]:
print ("\nTrain Model Report")
print ("Accuracy : %.4g" % metrics.accuracy_score(y_train, X_train_predictions))
print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_train, X_train_predprob))

print ("\nTest Model Report")
print ("Accuracy : %.4g" % metrics.accuracy_score(y_test, X_test_predictions))
print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_test, X_test_predprob))

feat_imp=pd.Series(xg_cl.get_booster().get_fscore()).sort_values(ascending=False)
#feat_imp
#feat_imp.plot(kind='bar', title='Feature Importances')
#plt.ylabel('Feature Importance Score')



Train Model Report
Accuracy : 0.9465
AUC Score (Train): 0.992969

Test Model Report
Accuracy : 0.939
AUC Score (Train): 0.980129


In [11]:
def predict(probs, threshold):
    preds = []
    for prob in probs:
        if prob > threshold:
            preds.append(1)
        else:
            preds.append(0)
    return np.array(preds)

f1_sc = 0
max_step = 0
for thr_step in np.linspace(0, 1, 101):
    _preds = predict(X_test_predprob, thr_step)
    f1_sc_step = f1_score(y_test, _preds)
    #print(thr_step, f1_sc_step)
    if f1_sc_step >= f1_sc:
        f1_sc = f1_sc_step
        max_step = thr_step
threshold = max_step
print('threshold', threshold)

threshold 0.81


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
def pretty_table(matrix):
    matrix_aux = chain.from_iterable(
        zip_longest(
            *(x.splitlines() for x in y),
            fillvalue='')
        for y in [[str(e) for e in row] for row in matrix])

    s = [[str(e) for e in row] for row in matrix_aux]
    lens = [max(map(len, col)) for col in zip(*s)]
    fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
    table = [fmt.format(*row) for row in s]
    print('\n'.join(table))
    
test_pred  = predict(X_test_predprob, threshold)
train_pred  = predict(X_train_predprob, threshold)
acc_train = metrics.accuracy_score(y_train, train_pred)
acc_test = metrics.accuracy_score(y_test, test_pred)

cm_train =  metrics.confusion_matrix(y_train, train_pred)
cm_test =  metrics.confusion_matrix(y_test, test_pred)

cm_train_pct = cm_train / cm_train.astype(np.float).sum()*100
cm_test_pct = cm_test / cm_test.astype(np.float).sum()*100

summary = [
    ['------', 'Train', 'Test'],
    ['confusion matrix', cm_train, cm_test],
    ['confusion matrix pct', cm_train_pct, cm_test_pct],
    ['f1 score', f1_score(y_train, train_pred), f1_score(y_test, test_pred)]#,
  #  ['accuracy score', metrics.accuracy_score(y_train, train_pred), metrics.accuracy_score(y_test, test_pred)],
   # ['recall score', metrics.recall_score(y_train, train_pred), metrics.recall_score(y_test, test_pred)],
    #['precision score', metrics.precision_score(y_train, train_pred), metrics.precision_score(y_test, test_pred)]
]

pretty_table(summary)


------              	Train                      	Test                       
confusion matrix    	[[322276   4571]           	[[80305  1407]             
                    	 [  1886  11496]]          	 [  929  2417]]            
confusion matrix pct	[[94.72325992  1.34350687] 	[[94.41204825  1.65416539] 
                    	 [ 0.55433252  3.37890068]]	 [ 1.09219591  2.84159044]]
f1 score            	0.7807395836870521         	0.6741980474198047         
